In [1]:
# import the necessary libraries
import argparse
import datetime
import time
import logging
import math
import os
import sys
import random
import datasets
import torch
from torch.optim import AdamW
from datasets import load_dataset, load_from_disk, concatenate_datasets
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import transformers
from accelerate import Accelerator, DistributedType
from accelerate.logging import get_logger
from accelerate.utils import set_seed, broadcast_object_list
from transformers import (
    BertConfig,
    BertTokenizerFast,
    BertForPreTraining,
    DataCollatorForLanguageModeling,
    DataCollatorWithPadding,
    SchedulerType,
    get_scheduler,
)
from transformers.utils.versions import require_version
from selectionstrategies import SubmodStrategy
from accelerate import InitProcessGroupKwargs
from selectionstrategies.helper_fns import taylor_softmax_v1
import numpy as np
import pickle
import faiss

os.environ['TOKENIZERS_PARALLELISM']='true'
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# Create a logger
logger = logging.getLogger(__name__)

# Set the logging level
logger.setLevel(logging.INFO)

# Create a file handler
file_handler = logging.FileHandler('logfile.log')

# Create a formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Set the formatter for the file handler
file_handler.setFormatter(formatter)

# Add the file handler to the logger
logger.addHandler(file_handler)

# Log a message
logger.info('Logger created')

In [3]:
# Variables 
dataset_name = "Salesforce/wikitext"
dataset_config_name = "wikitext-2-raw-v1"
validation_split_percentage = 80
model_config_name = "google-bert/bert-base-uncased"
tokenizer_name = "bert-base-uncased"
use_slow_tokenizer = False  # Bool
num_workers = None # (int)
max_seq_len = 128
short_seq_prob = 0.1
nsp_probability = 0.1
batch_size = 1000

In [4]:
# Get and Preprocess the dataset for the task.
raw_datasets = load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1")

if 'validation' not in raw_datasets.keys():
    raw_datasets=raw_datasets["train"].train_test_split(test_size=(validation_split_percentage/100), shuffle=False)
    raw_datasets=datasets.DatasetDict({"train": raw_datasets["train"], "validation": raw_datasets["test"]})

In [5]:
raw_datasets

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [6]:
# # Uncomment to use custom config

# config = BertConfig(
#     vocab_size=vocab_size,
#     hidden_size=hidden_size,
#     num_hidden_layers=num_hidden_layers,
#     num_attention_heads=num_attention_heads,
#     intermediate_size=intermediate_size,
#     hidden_act="gelu",
#     hidden_dropout_prob=0.1,
#     attention_probs_dropout_prob=0.1,
#     max_position_embeddings=512,
#     type_vocab_size=2,
#     initializer_range=0.02,
#     layer_norm_eps=1e-12,
#     position_embedding_type="absolute",
# )

In [7]:
# Create and instance of the model along with its tokenizer

# Tokenizer
tokenizer = BertTokenizerFast.from_pretrained(tokenizer_name, use_fast= not use_slow_tokenizer)

# Load the model
config = BertConfig.from_pretrained(model_config_name)

# Instantiating the model
model = BertForPreTraining(config)

# Resizing the token embeddings to fit the tokenizer
model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

In [8]:
# Tokenize and group the data based on the kind of model

column_names=raw_datasets['train'].column_names
text_column_name="text" if "text" in column_names else column_names[0]

def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

tokenized_dataset = raw_datasets.map(
    tokenize_function,
    batched=True,
    num_proc=num_workers, 
    remove_columns=column_names,
    desc="Running tokenizer on every text in dataset"
)

# Grouping the data 
from experiment_utils import group_texts

train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["validation"]

train_dataset = train_dataset.map(
    group_texts, 
    fn_kwargs={'split': 'train', 'tokenizer':tokenizer, 'max_seq_length': max_seq_len, 
               'short_seq_prob':short_seq_prob, 'nsp_probability':nsp_probability, 'tokenized_datasets':tokenized_dataset},
    batched=True,
    batch_size=batch_size,
    num_proc=num_workers,
    with_indices=True,
    desc=f"Grouping Train texts into chucks of {max_seq_len}"
)

eval_dataset = eval_dataset.map(
    group_texts, 
    fn_kwargs={'split': 'validation', 'tokenizer':tokenizer, 'max_seq_length': max_seq_len, 
               'short_seq_prob':short_seq_prob, 'nsp_probability':nsp_probability, 'tokenized_datasets':tokenized_dataset},
    batched=True,
    batch_size=batch_size,
    num_proc=num_workers,
    with_indices=True,
    desc=f"Grouping Validation texts into chucks of {max_seq_len}"
)

In [9]:
eval_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask', 'next_sentence_label'],
    num_rows: 1196
})

In [10]:
tokenizer.decode(eval_dataset[9]['input_ids'])

'[CLS] lobster pots, although lines baited with octopus or cuttlefish sometimes succeed in tempting them out, to allow them to be caught in a net or by hand. in 2008, 4 @, @ 386 t of h. gammarus were caught across europe and north africa, of which 3 @, @ 462 t ( 79 % ) was caught in the british isles ( including the channel islands ). the minimum landing size for h. gammarus is a carapace length of 87 mm ( 3 @. @ 4 [SEP] aquaculture systems for h. gammarus are under development, and production rates are still very low. [SEP]'

In [11]:
# Prepare the data
prepared_data = datasets.DatasetDict({"train": train_dataset, "validation": eval_dataset})
dataset=prepared_data['train']

def extract_first_sentences(examples):
    for i, input_ids in enumerate(examples["input_ids"]):
        idx=input_ids.index(tokenizer.sep_token_id)
        examples["input_ids"][i]=input_ids[:idx+1]
        examples["attention_mask"][i]=examples["attention_mask"][i][:idx+1]
        examples["token_type_ids"][i]=examples["token_type_ids"][i][:idx+1]
        examples["special_tokens_mask"][i]=examples["special_tokens_mask"][i][:idx+1]
    return examples

# Separate the data into those that have the next sentence labels and those that do not.
nsp_zero=dataset.filter(lambda examples: [x==0 for x in examples["next_sentence_label"]], batched=True, num_proc=num_workers, keep_in_memory=True)
nsp_one=dataset.filter(lambda examples: [x==1 for x in examples["next_sentence_label"]], batched=True, num_proc=num_workers, keep_in_memory=True)

# Extract the first sentences from both datasets
first_sent_nsp_zero=nsp_zero.map(extract_first_sentences, batched=True, num_proc=num_workers, remove_columns=["next_sentence_label", "special_tokens_mask"], keep_in_memory=True)
first_sent_nsp_one=nsp_one.map(extract_first_sentences, batched=True, num_proc=num_workers, remove_columns=["next_sentence_label", "special_tokens_mask"], keep_in_memory=True)

Filter:   0%|          | 0/12198 [00:00<?, ? examples/s]

Filter:   0%|          | 0/12198 [00:00<?, ? examples/s]

Map:   0%|          | 0/6467 [00:00<?, ? examples/s]

Map:   0%|          | 0/5731 [00:00<?, ? examples/s]

In [12]:
tokenizer.decode(first_sent_nsp_one[0]['input_ids'])

'[CLS] of movement limited by their action gauge. up to nine characters can be assigned to a single mission. during gameplay, characters will call out if something happens to them, such as their health points ( hp ) getting low or being knocked out by enemy attacks. each character has specific " potentials ", skills unique to each character. they are divided into " personal potential ", which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede a character, and " battle potentials ", which are grown throughout the game and always grant boons to a character. to learn battle [SEP]'

In [13]:
subset_fraction = 0.25

# Sample a subset of the train dataset
num_samples = int(round(len(train_dataset) * subset_fraction, 0))
init_subset_indices = [random.sample(list(range(len(train_dataset))), num_samples)]
full_dataset=train_dataset
subset_dataset = full_dataset.select(init_subset_indices[0])

In [14]:
first_sent_nsp_zero

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6467
})

In [15]:
train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask', 'next_sentence_label'],
    num_rows: 12198
})

In [16]:
subset_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask', 'next_sentence_label'],
    num_rows: 3050
})

In [17]:
mlm_probability = 0.15
train_batch_size = 4
eval_batch_size = 4

# Create datacollators
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=mlm_probability)
data_collator_embd = DataCollatorWithPadding(tokenizer=tokenizer)

# warmstart dataloader (train on all the train dataset during warmup)
warmstart_dataloader = DataLoader(train_dataset.remove_columns(['special_tokens_mask']), shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)

# first sent nsp zero dataloader
first_sent_nsp_zero_dataloader=DataLoader(first_sent_nsp_zero, shuffle=False, collate_fn=data_collator_embd, batch_size=eval_batch_size)

# first sent nsp one  dataloader
first_sent_nsp_one_dataloader=DataLoader(first_sent_nsp_one, shuffle=False, collate_fn=data_collator_embd, batch_size=eval_batch_size)

# subset dataloader (train)
subset_dataloader=DataLoader(subset_dataset.remove_columns(['special_tokens_mask']), shuffle=True, collate_fn=data_collator, batch_size=train_batch_size,)

# eval dataloader (validation & testing)
eval_dataloader=DataLoader(eval_dataset.remove_columns(['special_tokens_mask']), collate_fn=data_collator, batch_size=eval_batch_size)

In [18]:
learning_rate = 1e-4
scheduler_name = 'linear'
num_warmup_steps = 10
num_training_steps = 10

# Optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

# learning scheduler
lr_scheduler = get_scheduler(
    name=scheduler_name,
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

In [19]:
warmer = iter(eval_dataloader)
X= next(warmer)
X['input_ids'].shape

torch.Size([4, 128])

In [21]:
warmstart_epochs = 1
completed_steps = 0

model.to(device)
# Warmstart the model: Train the model with the warmstart data for warmstart epochs
for epoch in range(warmstart_epochs):
    if epoch==0:
        print("Begining warmstart")
    model.train() # Setup the model for training
    for step, batch in enumerate(warmstart_dataloader):
        outputs = model(**batch.to(device))
        loss = outputs.loss
        print(f"Completed Steps: {1+completed_steps}; Loss: {loss.detach().float()}; lr: {lr_scheduler.get_last_lr()};")
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        completed_steps += 1
        if completed_steps >= num_warmup_steps:
            break

    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():  # Use torch.no_grad() instead of inference_mode()
            outputs = model(**batch)
        
        loss = outputs.loss
        losses.append(loss.view(1))  # Add the loss as a 1-dimensional tensor

    if losses:
        losses = torch.cat(losses)
        losses = losses[:len(eval_dataset)]
        try:
            perplexity = math.exp(torch.mean(losses).item())
        except OverflowError:
            perplexity = float("inf")
    else:
        perplexity = float("inf")
    
    print(f"Epoch {epoch + 1}: Perplexity: {perplexity}")

# Plot both training & perplexity


Begining warmstart
Completed Steps: 1; Loss: 11.097604751586914; lr: [0.0];
Completed Steps: 2; Loss: 11.282768249511719; lr: [1e-05];
Completed Steps: 3; Loss: 11.085468292236328; lr: [2e-05];
Completed Steps: 4; Loss: 11.099989891052246; lr: [3e-05];
Completed Steps: 5; Loss: 10.559730529785156; lr: [4e-05];
Completed Steps: 6; Loss: 10.650423049926758; lr: [5e-05];
Completed Steps: 7; Loss: 10.43998908996582; lr: [6e-05];
Completed Steps: 8; Loss: 9.842432022094727; lr: [7e-05];
Completed Steps: 9; Loss: 10.256096839904785; lr: [8e-05];
Completed Steps: 10; Loss: 9.900712013244629; lr: [9e-05];


In [22]:
selection_strategy = 'fl'
num_partitions = 2000 # Default is 5000
partition_strategy = 'random'
submod_optimizer = 'LazyGreedy'

# Define subset selection strategies

subset_strategy = SubmodStrategy(logger, selection_strategy,
    num_partitions=num_partitions, partition_strategy=partition_strategy,
    optimizer=submod_optimizer, similarity_criterion='feature', 
    metric='cosine', eta=1, stopIfZeroGain=False, 
    stopIfNegativeGain=False, verbose=False, lambdaVal=1)

In [24]:
selection_strategy = 'fl'
layer_for_similarity_computation = 9
temperature = 0.5
seed = 23
parallel_processes = 3
# num_samples has already been defined when creating subset.
probs_nsp_zero=[]
greedyList_nsp_zero=[]
gains_nsp_zero=[]


# Begin subset selection for first_sent_nsp_zero
if selection_strategy == 'Random-Online':
    subset_indices_nsp_zero = [random.sample(list(range(len(first_sent_nsp_zero))), math.floor(num_samples/2))]
    subset_indices_nsp_one = [random.sample(list(range(len(first_sent_nsp_one))), math.ceil(num_samples/2))]
elif selection_strategy in ['fl', 'logdet', 'gc', 'disparity-sum']:
    # Choose a selection strategy
    model.eval() # Set the model in evaluation model 
    representations_nsp_zero=[]
    batch_indices_nsp_zero=[]
    total_cnt=0
    total_storage=0
    # Unwrap the model and set it in evaluation mode.
    print("Performing Subset selection for NSP class 0")
    for step, batch in enumerate(first_sent_nsp_zero_dataloader):
        with torch.no_grad():
            output = model(**batch.to(device), output_hidden_states=True)
        embeddings=output["hidden_states"][layer_for_similarity_computation]
        # print(f"Embeddings shape: {embeddings.shape}")
        mask=(batch['attention_mask'].unsqueeze(-1).expand(embeddings.size()).float())
        mask1=((batch['token_type_ids'].unsqueeze(-1).expand(embeddings.size()).float())==0)
        mask=mask*mask1
        mean_pooled=torch.sum(embeddings*mask, 1) / torch.clamp(mask.sum(1), min=1e-9)
        total_cnt += mean_pooled.size(0)
        mean_pooled = mean_pooled.cpu()
        total_storage += sys.getsizeof(mean_pooled.storage())
        representations_nsp_zero.append(mean_pooled)
        # print(f"Current total representations: {len(representations_nsp_zero)}")
        
    print(f"Final number of representations: {len(representations_nsp_zero)}")
    representations_nsp_zero=torch.cat(representations_nsp_zero, dim=0)
    representations_nsp_zero=representations_nsp_zero[:len(first_sent_nsp_zero)]
    total_storage += sys.getsizeof(representations_nsp_zero.storage())
    representations_nsp_zero=representations_nsp_zero.numpy()
    print('Representations(NSP Class 0) Size: {}, Total number of samples: {}'.format(total_storage/(1024 * 1024), total_cnt))
    batch_indices_nsp_zero=list(range(len(first_sent_nsp_zero)))
    print('Length of indices: {}'.format(len(batch_indices_nsp_zero)))
    print('Representations(NSP Class 0) gathered. Shape of representations: {}. Length of indices: {}'.format(representations_nsp_zero.shape, len(batch_indices_nsp_zero)))

    partition_indices_nsp_zero, greedyIdx_nsp_zero, gains_nsp_zero = subset_strategy.select(len(batch_indices_nsp_zero)-1, 
                                                                                            batch_indices_nsp_zero, representations_nsp_zero, 
                                                                                            parallel_processes=parallel_processes, return_gains=True)
    subset_indices_nsp_zero = [[]]
    i=0
    for p in gains_nsp_zero:
        greedyList_nsp_zero.append(greedyIdx_nsp_zero[i:i+len(p)])         
        i+=len(p)
    probs_nsp_zero=[taylor_softmax_v1(torch.from_numpy(np.array([partition_gains])/temperature)).numpy()[0] for partition_gains in gains_nsp_zero]
    print(f"Taylor Softmax Prop: {probs_nsp_zero}")
    rng=np.random.default_rng(seed+completed_steps)
    for i, partition_prob in enumerate(probs_nsp_zero):
        print(f"{i}: Partition probablity :{partition_prob}")
        partition_budget=min(math.ceil((len(partition_prob)/len(batch_indices_nsp_zero)) * math.floor(num_samples/2)), len(partition_prob)-1)
        subset_indices_nsp_zero[0].extend(rng.choice(greedyList_nsp_zero[i], size=partition_budget, replace=False, p=partition_prob).tolist())

nsp_zero_subset_dataset=nsp_zero.select(subset_indices_nsp_zero[0])

# Using the list and the selection strategy, get the indices and the gains of each data point in the list.

Performing Subset selection for NSP class 0
Final number of representations: 1617
Representations(NSP Class 0) Size: 37.966644287109375, Total number of samples: 6467
Length of indices: 6467
Representations(NSP Class 0) gathered. Shape of representations: (6467, 768). Length of indices: 6467


100%|██████████| 1617/1617 [00:00<00:00, 2199.76it/s]eration %2 [Iteration  of 3 of 33]]]f % [Iteration 31] of 3]3]33% [Iteration 1 of 3]33% [Iteration 1 of 3]33% [Iteration 1 of 3]


Taylor Softmax Prop: [array([0.95152425, 0.02484734, 0.02362841]), array([0.95099156, 0.02482702, 0.02418142]), array([0.94490022, 0.02908357, 0.02601621]), array([0.94766461, 0.02690931, 0.02542608]), array([0.94557038, 0.02906514, 0.02536447]), array([0.94493821, 0.0294948 , 0.02556699]), array([0.94795698, 0.0269494 , 0.02509362]), array([0.94766435, 0.02699743, 0.02533822]), array([0.94722654, 0.02702508, 0.02574838]), array([0.94931659, 0.02674636, 0.02393705]), array([0.95313119, 0.02344654, 0.02342227]), array([0.95174117, 0.02462268, 0.02363615]), array([0.95015798, 0.02576666, 0.02407535]), array([0.95017662, 0.02577663, 0.02404674]), array([0.94506342, 0.03055871, 0.02437787]), array([0.94868245, 0.02692846, 0.02438909]), array([0.95291707, 0.02357324, 0.0235097 ]), array([0.95178775, 0.02418319, 0.02402906]), array([0.94974367, 0.0261767 , 0.02407964]), array([0.94933826, 0.02674955, 0.02391219]), array([0.95167458, 0.0243545 , 0.02397092]), array([0.952786  , 0.02374984, 0.

In [26]:
gains_nsp_one=[]
probs_nsp_one=[]
greedyList_nsp_one=[]

if selection_strategy in ['fl', 'logdet', 'gc', 'disparity-sum']:
    # Choose a selection strategy
    model.eval() # Set the model in evaluation model 
    representations_nsp_one=[]
    batch_indices_nsp_one=[]
    total_cnt=0
    total_storage=0
    # Unwrap the model and set it in evaluation mode.
    print("Performing Subset selection for NSP class 1")
    for step, batch in enumerate(first_sent_nsp_one_dataloader):
        with torch.no_grad():
            output = model(**batch.to(device), output_hidden_states=True)
        embeddings=output["hidden_states"][layer_for_similarity_computation]
        # print(f"Embeddings shape: {embeddings.shape}")
        mask=(batch['attention_mask'].unsqueeze(-1).expand(embeddings.size()).float())
        mask1=((batch['token_type_ids'].unsqueeze(-1).expand(embeddings.size()).float())==0)
        mask=mask*mask1
        mean_pooled=torch.sum(embeddings*mask, 1) / torch.clamp(mask.sum(1), min=1e-9)
        total_cnt += mean_pooled.size(0)
        mean_pooled = mean_pooled.cpu()
        total_storage += sys.getsizeof(mean_pooled.storage())
        representations_nsp_one.append(mean_pooled)
        # print(f"Current total representations: {len(representations_nsp_one)}")
        
    print(f"Final number of representations: {len(representations_nsp_one)}")
    representations_nsp_one=torch.cat(representations_nsp_one, dim=0)
    representations_nsp_one=representations_nsp_one[:len(first_sent_nsp_one)]
    total_storage += sys.getsizeof(representations_nsp_one.storage())
    representations_nsp_one=representations_nsp_one.numpy()
    print('Representations(NSP Class 1) Size: {}, Total number of samples: {}'.format(total_storage/(1024 * 1024), total_cnt))
    batch_indices_nsp_one=list(range(len(first_sent_nsp_one)))
    print('Length of indices: {}'.format(len(batch_indices_nsp_one)))
    print('Representations(NSP Class 0) gathered. Shape of representations: {}. Length of indices: {}'.format(representations_nsp_one.shape, len(batch_indices_nsp_one)))

    partition_indices_nsp_one, greedyIdx_nsp_one, gains_nsp_one = subset_strategy.select(len(batch_indices_nsp_one)-1, 
                                                                                            batch_indices_nsp_one, representations_nsp_one, 
                                                                                            parallel_processes=parallel_processes, return_gains=True)
    subset_indices_nsp_one = [[]]
    i=0
    for p in gains_nsp_one:
        greedyList_nsp_one.append(greedyIdx_nsp_one[i:i+len(p)])         
        i+=len(p)
    probs_nsp_one=[taylor_softmax_v1(torch.from_numpy(np.array([partition_gains])/temperature)).numpy()[0] for partition_gains in gains_nsp_one]
    print(f"Taylor Softmax Prop: {probs_nsp_one}")
    rng=np.random.default_rng(seed+completed_steps)
    for i, partition_prob in enumerate(probs_nsp_one):
        print(f"{i}: Partition probablity :{partition_prob}")
        if len(partition_prob) > 0:
            partition_budget=min(math.ceil((len(partition_prob)/len(batch_indices_nsp_one)) * math.floor(num_samples/2)), len(partition_prob)-1)
            print(f"Partition Budget: {partition_budget}")
            subset_indices_nsp_one[0].extend(rng.choice(greedyList_nsp_one[i], size=partition_budget, replace=False, p=partition_prob).tolist())

nsp_one_subset_dataset=nsp_one.select(subset_indices_nsp_one[0])

Performing Subset selection for NSP class 1
Final number of representations: 1433
Representations(NSP Class 1) Size: 33.645721435546875, Total number of samples: 5731
Length of indices: 5731
Representations(NSP Class 0) gathered. Shape of representations: (5731, 768). Length of indices: 5731


100%|██████████| 1911/1911 [00:00<00:00, 2670.84it/s]ion 1 of 2]2f 2] of 22]] 1 of 2]] 2]] [Iteration 1 of 502%] [Iteration 1 of 2]% [Iteration 1 of 2]2]


Taylor Softmax Prop: [array([0.9613222, 0.0386778]), array([0.96131414, 0.03868586]), array([0.96135111, 0.03864889]), array([0.9613197, 0.0386803]), array([0.96124186, 0.03875814]), array([0.96136933, 0.03863067]), array([0.96134181, 0.03865819]), array([0.96138984, 0.03861016]), array([0.96132782, 0.03867218]), array([0.96134339, 0.03865661]), array([0.96140466, 0.03859534]), array([0.96141227, 0.03858773]), array([0.96133379, 0.03866621]), array([0.96136198, 0.03863802]), array([0.96141329, 0.03858671]), array([0.96129822, 0.03870178]), array([0.96133421, 0.03866579]), array([0.96131878, 0.03868122]), array([0.96134865, 0.03865135]), array([0.96135744, 0.03864256]), array([0.95557736, 0.04442264]), array([0.95944086, 0.04055914]), array([0.9613157, 0.0386843]), array([0.96135903, 0.03864097]), array([0.96131648, 0.03868352]), array([0.96132197, 0.03867803]), array([0.96136243, 0.03863757]), array([0.95859704, 0.04140296]), array([0.9613383, 0.0386617]), array([0.96137024, 0.03862976

In [27]:
subset_indices_nsp_one

[[5094,
  200,
  1207,
  2999,
  1858,
  4022,
  3219,
  2004,
  707,
  857,
  1291,
  2522,
  4258,
  3419,
  5696,
  4393,
  5175,
  1354,
  884,
  2878,
  1621,
  2766,
  400,
  3064,
  4769,
  4888,
  2015,
  2169,
  1135,
  5087,
  2872,
  2729,
  1699,
  1024,
  5536,
  890,
  2465,
  5354,
  3003,
  3404,
  5121,
  972,
  4554,
  4966,
  3008,
  2108,
  269,
  1684,
  5090,
  4263,
  695,
  2077,
  892,
  2154,
  5120,
  474,
  4629,
  5561,
  5668,
  581,
  2388,
  807,
  4511,
  964,
  4640,
  3453,
  2658,
  2866,
  3357,
  3523,
  568,
  1525,
  2850,
  1857,
  2742,
  2785,
  5529,
  1122,
  1958,
  3144,
  528,
  2340,
  4157,
  3686,
  4984,
  3790,
  3909,
  1509,
  4597,
  1987,
  1342,
  4277,
  2488,
  289,
  52,
  329,
  3581,
  3369,
  3931,
  4487,
  5487,
  2807,
  299,
  5627,
  1111,
  5432,
  4993,
  2001,
  4520,
  5174,
  927,
  5599,
  3675,
  5473,
  4817,
  1711,
  5665,
  2211,
  2342,
  3189,
  5011,
  5061,
  5033,
  3648,
  1570,
  4607,
  3388,
  333,

In [28]:
partition_budget

1

In [29]:
greedyList_nsp_zero

[[122, 4376, 5370],
 [4561, 1051, 5868],
 [2747, 2826, 4689],
 [5034, 2981, 1044],
 [673, 3467, 5147],
 [5294, 2391, 181],
 [5631, 2512, 3462],
 [3346, 3929, 2366],
 [14, 1358, 710],
 [573, 5334, 691],
 [3446, 2445, 2599],
 [5512, 2639, 4562],
 [5709, 3888, 4295],
 [3111, 3041, 5102],
 [4368, 1570, 4060],
 [6167, 1670, 3173],
 [4617, 2361, 6429],
 [5308, 1927, 4709],
 [5435, 5856, 5766],
 [5579, 4089, 3440],
 [2210, 2276, 4647],
 [1355, 1912, 6335],
 [5876, 4394, 4990],
 [5461, 2912, 219],
 [4114, 2892, 4157],
 [6204, 3079, 4502],
 [3268, 5114, 3607],
 [2097, 1282, 2064],
 [4477, 1028, 4183],
 [965, 5401, 1494],
 [1256, 2175, 5693],
 [5684, 948, 1746],
 [1823, 6094, 2885],
 [3724, 63, 1590],
 [1251, 2876, 749],
 [1527, 6043, 5041],
 [3054, 5544, 5378],
 [3651, 285, 4388],
 [1260, 4595, 6341],
 [2270, 4363, 407],
 [67, 5958, 4688],
 [5538, 5743, 5602],
 [2244, 4194, 3123],
 [1757, 5840, 4444],
 [4839, 2804, 5548],
 [6200, 134, 1083],
 [5973, 2117, 1763],
 [4454, 5, 982],
 [2204, 4938, 3

In [30]:
# Put all the data into a dataset called subset_dataset
# Concatenate the two datasets
subset_dataset = concatenate_datasets([nsp_zero_subset_dataset, nsp_one_subset_dataset])

subset_dataloader=DataLoader(
    subset_dataset.remove_columns(['special_tokens_mask']), shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)

# add the data to the subset dataloader

In [31]:
subset_dataset.remove_columns(['special_tokens_mask'])

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'next_sentence_label'],
    num_rows: 3527
})

In [34]:
max_train_steps = 10
completed_steps = 0 # I needed to reset this!
select_every = 5 # This is needed to breaking the training and set the subset selection back in motion.
print("Begin the main training loop with importance re-sampling, after warm-start")
while completed_steps<max_train_steps:
    model.train()
    select_subset=False
    for step, batch in enumerate(subset_dataloader):
        train_time=0
        subset_time=0
        start_time=time.time()
        outputs=model(**batch.to(device))
        loss=outputs.loss
        logger.info(f"Completed Steps: {1+completed_steps}; Loss: {loss.detach().float()}; lr: {lr_scheduler.get_last_lr()};")
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        completed_steps+=1
           
        train_time += (time.time() - start_time)

        if completed_steps>=max_train_steps:
            break
        
        if (completed_steps)%select_every==0:
            select_subset=True
            break

    if select_subset==True:
        start_time = time.time()
        num_samples = int(round(len(full_dataset) * subset_fraction, 0)) 
        if selection_strategy == 'Random-Online':
            subset_indices_nsp_zero = [random.sample(list(range(len(first_sent_nsp_zero))), math.floor(num_samples/2))]
            subset_indices_nsp_one = [random.sample(list(range(len(first_sent_nsp_one))), math.ceil(num_samples/2))]

        elif selection_strategy in ["fl", "logdet", "gc", "disparity-min"]:
            print(f"Performing Subset selection for NSP class 0")
            sampling_start_time=time.time()
            
            subset_indices_nsp_zero=[[]]
            rng=np.random.default_rng(seed+completed_steps)
            for i, partition_prob in enumerate(probs_nsp_zero):
                partition_budget=min(math.ceil((len(partition_prob)/len(batch_indices_nsp_zero)) * math.floor(num_samples/2)), len(partition_prob)-1)
                subset_indices_nsp_zero[0].extend(rng.choice(greedyList_nsp_zero[i], size=partition_budget, replace=False, p=partition_prob).tolist())

            print("Sampling time(NSP Class 0): {}".format(time.time()-sampling_start_time))
        
            logger.info(f"Performing Subset selection for NSP class 1")
            sampling_start_time=time.time()
            
            subset_indices_nsp_one=[[]]
            rng=np.random.default_rng(seed+completed_steps)
            for i, partition_prob in enumerate(probs_nsp_one):
                if len(partition_prob) > 0:
                    partition_budget=min(math.ceil((len(partition_prob)/len(batch_indices_nsp_one)) * math.ceil(num_samples/2)), len(partition_prob)-1)
                    subset_indices_nsp_one[0].extend(rng.choice(greedyList_nsp_one[i], size=partition_budget, replace=False, p=partition_prob).tolist())

            print("Sampling time(NSP Class 1): {}".format(time.time()-sampling_start_time))

        nsp_zero_subset_dataset=nsp_zero.select(subset_indices_nsp_zero[0])
        nsp_one_subset_dataset=nsp_one.select(subset_indices_nsp_one[0])
        
        # Concatenate the two datasets
        subset_dataset = concatenate_datasets([nsp_zero_subset_dataset, nsp_one_subset_dataset])
        subset_dataloader=DataLoader(
            subset_dataset.remove_columns(['special_tokens_mask']), shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)

        select_subset=False # Setting the up for training after re-sampling

        print(f"Subset selection Completed")
    
    model.eval()
    losses=[]
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs=model(**batch.to(device))

        loss=outputs.loss
        losses.append(loss.view(1))

    losses=torch.cat(losses)
    losses=losses[:len(eval_dataset)]
    try:
        perplexity=math.exp(torch.mean(losses))
    except OverflowError:
        perplexity=float("inf")

    print(f"Steps {completed_steps}: perplexity: {perplexity}")

print(f"Saving the final model after {completed_steps} steps.")
print(f"Training completed successfully!")

Begin the main training loop with importance re-sampling, after warm-start
Performing Subset selection for NSP class 0
Sampling time(NSP Class 0): 0.06132841110229492
Sampling time(NSP Class 1): 0.06155204772949219
Subset selection Completed
Steps 5: perplexity: 20640.785199319973
Steps 10: perplexity: 20701.14749444851
Saving the final model after 10 steps.
Training completed successfully!


In [35]:
# Train with importance re-sampling

# Train on the entire dataset once

# Sample using the indices and gains

# Train the model on the sampled dataset

# Evaluate the model 

# Save the model 

In [36]:
# Personal Addition: Inference on the model.